# Wine Quality — Random Forest Regression (one command per code cell)
# Ensemble of decision trees; robust to nonlinearities and interactions.


# Imports

In [ ]:
import pandas as pd  # data handling


In [ ]:
import numpy as np  # numeric helpers


In [ ]:
from sklearn.model_selection import train_test_split, cross_validate  # split and CV


In [ ]:
from sklearn.ensemble import RandomForestRegressor  # random forest regressor


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score  # metrics


# Load dataset

In [ ]:
DATASET = 'red'  # 'red' or 'white'


In [ ]:
CSV_PATH = f"/Users/sb/Documents/Study-Material/Machine-Learning/winequality-{DATASET}.csv"  # dataset path


In [ ]:
df = pd.read_csv(CSV_PATH, sep=';')  # load CSV


# Split

In [ ]:
X = df.drop(columns=['quality'])  # features


In [ ]:
y = df['quality']  # target


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # split


# Model

In [ ]:
rf = RandomForestRegressor(n_estimators=300, max_depth=None, random_state=42, n_jobs=None)  # random forest parameters


In [ ]:
rf.fit(X_train, y_train)  # fit model


In [ ]:
pred = rf.predict(X_test)  # predictions


# Evaluate

In [ ]:
mae = mean_absolute_error(y_test, pred)  # MAE


In [ ]:
rmse = mean_squared_error(y_test, pred, squared=False)  # RMSE


In [ ]:
r2 = r2_score(y_test, pred)  # R^2


In [ ]:
{'MAE': mae, 'RMSE': rmse, 'R2': r2}  # metrics


# Cross-validation

In [ ]:
cv = cross_validate(rf, X, y, cv=5, scoring=('r2','neg_root_mean_squared_error'))  # 5-fold CV


In [ ]:
{k: (np.mean(v), np.std(v)) for k, v in cv.items() if k.startswith('test_')}  # CV summary


# Feature importance (optional quick look)

In [ ]:
dict(sorted({col: imp for col, imp in zip(X.columns, rf.feature_importances_)}.items(), key=lambda kv: kv[1], reverse=True))  # importance by feature


# MLflow Tracking — log params, metrics, and model

In [ ]:
import mlflow  # experiment tracking


In [ ]:
import mlflow.sklearn  # sklearn flavor logging


In [ ]:
mlflow.set_experiment('RandomForestRegression-WineQuality')  # experiment per algorithm


In [ ]:
mlflow.start_run(run_name=f"{DATASET}")  # start run


In [ ]:
mlflow.log_params({'n_estimators': getattr(rf, 'n_estimators', None), 'max_depth': getattr(rf, 'max_depth', None), 'random_state': getattr(rf, 'random_state', None)})  # params


In [ ]:
mlflow.log_metric('MAE', mae)  # MAE


In [ ]:
mlflow.log_metric('RMSE', rmse)  # RMSE


In [ ]:
mlflow.log_metric('R2', r2)  # R2


In [ ]:
mlflow.sklearn.log_model(rf, artifact_path='model')  # log model


In [ ]:
mlflow.end_run()  # end run


# Persistence — save to disk and reload

In [ ]:
import os  # fs utils


In [ ]:
import joblib  # persistence


In [ ]:
from datetime import datetime  # timestamp


In [ ]:
os.makedirs('Machine-Learning/models/RandomForestRegression', exist_ok=True)  # ensure dir


In [ ]:
MODEL_PATH = f"Machine-Learning/models/RandomForestRegression/random_forest_wine_{DATASET}_" + datetime.now().strftime('%Y%m%d_%H%M%S') + ".joblib"  # path


In [ ]:
joblib.dump(rf, MODEL_PATH)  # save


In [ ]:
loaded = joblib.load(MODEL_PATH)  # load


In [ ]:
float(loaded.predict(X_test.iloc[[0]])[0])  # verify loaded model prediction


# Streamlit App — minimal predictor UI

In [ ]:
APP_CODE = """
import joblib, pandas as pd, streamlit as st
st.set_page_config(page_title='Wine — Random Forest', page_icon='🍷')
st.title('🍷 Wine Quality — Random Forest Regression')
MODEL_PATH = st.text_input('Model path', value=r'%s')
if MODEL_PATH and st.button('Load model'):
    try:
        st.session_state['model'] = joblib.load(MODEL_PATH)
        st.success('Model loaded')
    except Exception as e:
        st.error(f'Load failed: {e}')
if 'model' in st.session_state:
    cols = %s
    vals = {c: st.number_input(c, value=0.0) for c in cols}
    if st.button('Predict quality'):
        df = pd.DataFrame([vals])
        try:
            y = st.session_state['model'].predict(df)
            st.metric('Predicted quality', f"{float(y[0]):.3f}")
        except Exception as e:
            st.error(f'Predict failed: {e}')
""" % (MODEL_PATH, list(X.columns))


In [ ]:
open('app_streamlit_wine.py', 'w').write(APP_CODE)


In [ ]:
float(rf.predict(X_test.iloc[[0]])[0])  # single-row prediction
